In [1]:
import sys
print(sys.path)


['', '/opt/openrobots/lib/python3.6/site-packages', '/opt/openrobots/lib/python3.6/dist-packages', '/home/tlha/catkin_crocoddyl_ws/install/lib/python3/dist-packages', '/home/tlha/tiago_public_ws/install/lib/python2.7/dist-packages', '/opt/ros/melodic/lib/python2.7/dist-packages', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/home/tlha/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/tlha/.local/lib/python3.6/site-packages/IPython/extensions', '/home/tlha/.ipython']


In [2]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data

robot = example_robot_data.load('tiago')
robot_model = robot.model
q0 = robot_model.referenceConfigurations["tuck_arm"]

DT = 1e-3
T= 250
target = np.array([0.8, 0.9, 0]) 
display = crocoddyl.MeshcatDisplay(robot, 4, 4, False)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
print("referenceConfigurations:", q0)

referenceConfigurations: [ 0.15  0.2  -1.34 -0.2   1.94 -1.57  1.37  0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.
  1.    0.  ]


In [4]:
print("robot.q0 size:", robot.q0.shape[0])
print(robot.q0)

robot.q0 size: 50
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 1. 0.]


In [5]:
robot_model

Nb joints = 49 (nq=50,nv=48)
  Joint 0 universe: parent=0
  Joint 1 torso_lift_joint: parent=0
  Joint 2 arm_1_joint: parent=1
  Joint 3 arm_2_joint: parent=2
  Joint 4 arm_3_joint: parent=3
  Joint 5 arm_4_joint: parent=4
  Joint 6 arm_5_joint: parent=5
  Joint 7 arm_6_joint: parent=6
  Joint 8 arm_7_joint: parent=7
  Joint 9 hand_index_abd_joint: parent=8
  Joint 10 hand_index_virtual_1_joint: parent=9
  Joint 11 hand_index_flex_1_joint: parent=10
  Joint 12 hand_index_virtual_2_joint: parent=11
  Joint 13 hand_index_flex_2_joint: parent=12
  Joint 14 hand_index_virtual_3_joint: parent=13
  Joint 15 hand_index_flex_3_joint: parent=14
  Joint 16 hand_index_joint: parent=8
  Joint 17 hand_little_abd_joint: parent=8
  Joint 18 hand_little_virtual_1_joint: parent=17
  Joint 19 hand_little_flex_1_joint: parent=18
  Joint 20 hand_little_virtual_2_joint: parent=19
  Joint 21 hand_little_flex_2_joint: parent=20
  Joint 22 hand_little_virtual_3_joint: parent=21
  Joint 23 hand_little_flex_3_j

In [6]:
model = crocoddyl.ActionModelUnicycle()
data = model.createData()
model.costWeights = np.matrix([
    1,   # state weight
    1  # control weight
]).T

In [23]:
data.shape()

AttributeError: 'ActionDataUnicycle' object has no attribute 'shape'

In [9]:
# Create the cost functions
Mref = crocoddyl.FrameTranslation(robot_model.getFrameId("base_link"), target)
state = crocoddyl.StateMultibody(robot.model)
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref)
xRegCost = crocoddyl.CostModelState(state)
uRegCost = crocoddyl.CostModelControl(state)

In [11]:
xRegCost

In [13]:
# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1e2)
runningCostModel.addCost("stateReg", xRegCost, 1e-4)
runningCostModel.addCost("ctrlReg", uRegCost, 1e-7)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e5)
terminalCostModel.addCost("stateReg", xRegCost, 1e-4)
terminalCostModel.addCost("ctrlReg", uRegCost, 1e-7)

In [19]:
# Create the actuation model
actuationModel = crocoddyl.ActuationModelFull(state)

# Create the action model
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), DT)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel))


# Create the problem
q0 = robot_model.referenceConfigurations["tuck_arm"]
x0 = np.concatenate([q0, pinocchio.utils.zero(state.nv)])
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

# Solving it with the DDP algorithm
ddp.solve()

# Visualizing the solution in gepetto-viewer
display.displayFromSolver(ddp)

robot_data = robot_model.createData()
xT = ddp.xs[-1]
pinocchio.forwardKinematics(robot_model, robot_data, xT[:state.nq])
pinocchio.updateFramePlacements(robot_model, robot_data)
print('Finally reached = ', robot_data.oMf[robot_model.getFrameId("base_link")].translation.T)

Finally reached =  [0.     0.     0.0985]


In [20]:
x0.shape[0]

98

In [16]:
x0

array([ 0.15,  0.2 , -1.34, -0.2 ,  1.94, -1.57,  1.37,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  1.  ,  0.  ,  1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
        0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [17]:
display.robot.viewer.jupyter_cell()

In [18]:
xT

array([-1.50134559e-01,  1.96842665e-01, -1.31480550e+00, -1.91828409e-01,
        1.88102726e+00, -1.52378710e+00,  1.33146057e+00,  9.59197937e-05,
        1.88983410e-04,  4.22810380e-04,  3.43257540e-04,  2.74074671e-04,
        2.04212409e-04,  1.35895219e-04,  6.73657377e-05,  8.43812577e-05,
        1.89065094e-04,  4.20775896e-04,  3.42532960e-04,  2.73720543e-04,
        2.04126738e-04,  1.35913428e-04,  6.73092568e-05,  1.91056395e-04,
        4.87310940e-04,  3.95800515e-04,  3.16280219e-04,  2.35638618e-04,
        1.56916567e-04,  7.76849776e-05,  8.43812577e-05,  1.90328650e-04,
        4.69871924e-04,  3.82001369e-04,  3.05289152e-04,  2.27550606e-04,
        1.51536309e-04,  7.50281094e-05,  1.80378593e-04,  1.39766694e-04,
        1.15205008e-04,  7.74610100e-05,  3.85033776e-05,  8.43812577e-05,
       -3.16491313e-09, -8.68679570e-04,  1.00000000e+00,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00, -2.41690255e+00, -4.44277845e-03,
        1.06465478e-01,  